In [ ]:
!pip install transformers datasets

In [20]:
from transformers import DistilBertConfig, DistilBertTokenizerFast, DistilBertForSequenceClassification
MODEL_PATH='distilbert-base-uncased'
config = DistilBertConfig.from_pretrained(MODEL_PATH, num_labels=1)
tokenizer = DistilBertTokenizerFast.from_pretrained(MODEL_PATH)
model = DistilBertForSequenceClassification.from_pretrained(MODEL_PATH, config=config)

loading configuration file https://huggingface.co/distilbert-base-uncased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/23454919702d26495337f3da04d1655c7ee010d5ec9d77bdb9e399e00302c0a1.d423bdf2f58dc8b77d5f5d18028d7ae4a72dcfd8f468e81fe979ada957a8c361
Model config DistilBertConfig {
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "id2label": {
    "0": "LABEL_0"
  },
  "initializer_range": 0.02,
  "label2id": {
    "LABEL_0": 0
  },
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.8.2",
  "vocab_size": 30522
}

loading file https://huggingface.co/distilbert-base-uncased/resolve/main/vocab.txt from cache at /root/.cache/huggingface/transformer

In [21]:
import datasets
from datasets import load_dataset
stsb_train= load_dataset('glue','stsb', split="train")
stsb_validation = load_dataset('glue','stsb', split="validation")
stsb_validation=stsb_validation.shuffle(seed=42)
stsb_val= datasets.Dataset.from_dict(stsb_validation[:750])
stsb_test= datasets.Dataset.from_dict(stsb_validation[750:])

Reusing dataset glue (/root/.cache/huggingface/datasets/glue/stsb/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)
Reusing dataset glue (/root/.cache/huggingface/datasets/glue/stsb/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)
Loading cached shuffled indices for dataset at /root/.cache/huggingface/datasets/glue/stsb/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-e044e8a2f565152b.arrow


In [22]:
import pandas as pd
pd.DataFrame(stsb_train)

,idx,label,sentence1,sentence2
0,0,5.00,A plane is taking off.,An air plane is taking off.
1,1,3.80,A man is playing a large flute.,A man is playing a flute.
2,2,3.80,A man is spreading shreded cheese on a pizza.,A man is spreading shredded cheese on an uncoo...
3,3,2.60,Three men are playing chess.,Two men are playing chess.
4,4,4.25,A man is playing the cello.,A man seated is playing the cello.
...,...,...,...,...
5744,5744,0.00,Severe Gales As Storm Clodagh Hits Britain,Merkel pledges NATO solidarity with Latvia
5745,5745,0.00,Dozens of Egyptians hostages taken by Libyan t...,Egyptian boat crash death toll rises as more b...
5746,5746,0.00,President heading to Bahrain,President Xi: China to continue help to fight ...
5747,5747,0.00,"China, India vow to further bilateral ties",China Scrambles to Reassure Jittery Stock Traders


In [23]:
stsb_train.shape, stsb_val.shape, stsb_test.shape

((5749, 4), (750, 4), (750, 4))

In [24]:
enc_train = stsb_train.map(lambda e: tokenizer( e['sentence1'],e['sentence2'], padding=True, truncation=True), batched=True, batch_size=1000) 
enc_val =   stsb_val.map(lambda e: tokenizer( e['sentence1'],e['sentence2'], padding=True, truncation=True), batched=True, batch_size=1000) 
enc_test =  stsb_test.map(lambda e: tokenizer( e['sentence1'],e['sentence2'], padding=True, truncation=True), batched=True, batch_size=1000)  


Loading cached processed dataset at /root/.cache/huggingface/datasets/glue/stsb/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-96b9dfcfcfd5f983.arrow


In [25]:
import pandas as pd
pd.DataFrame(enc_train)

,attention_mask,idx,input_ids,label,sentence1,sentence2
0,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",0,"[101, 1037, 4946, 2003, 2635, 2125, 1012, 102,...",5.00,A plane is taking off.,An air plane is taking off.
1,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",1,"[101, 1037, 2158, 2003, 2652, 1037, 2312, 8928...",3.80,A man is playing a large flute.,A man is playing a flute.
2,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",2,"[101, 1037, 2158, 2003, 9359, 14021, 5596, 209...",3.80,A man is spreading shreded cheese on a pizza.,A man is spreading shredded cheese on an uncoo...
3,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",3,"[101, 2093, 2273, 2024, 2652, 7433, 1012, 102,...",2.60,Three men are playing chess.,Two men are playing chess.
4,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",4,"[101, 1037, 2158, 2003, 2652, 1996, 10145, 101...",4.25,A man is playing the cello.,A man seated is playing the cello.
...,...,...,...,...,...,...
5744,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",5744,"[101, 5729, 14554, 2015, 2004, 4040, 18856, 13...",0.00,Severe Gales As Storm Clodagh Hits Britain,Merkel pledges NATO solidarity with Latvia
5745,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",5745,"[101, 9877, 1997, 23437, 19323, 2579, 2011, 19...",0.00,Dozens of Egyptians hostages taken by Libyan t...,Egyptian boat crash death toll rises as more b...
5746,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",5746,"[101, 2343, 5825, 2000, 15195, 102, 2343, 8418...",0.00,President heading to Bahrain,President Xi: China to continue help to fight ...
5747,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",5747,"[101, 2859, 1010, 2634, 19076, 2000, 2582, 177...",0.00,"China, India vow to further bilateral ties",China Scrambles to Reassure Jittery Stock Traders


In [26]:
from transformers import TrainingArguments, Trainer

In [27]:
training_args = TrainingArguments(
    # The output directory where the model predictions and checkpoints will be written
    output_dir='./stsb-model', 
    do_train=True,
    do_eval=True,
    #  The number of epochs, defaults to 3.0 
    num_train_epochs=3,              
    per_device_train_batch_size=32,  
    per_device_eval_batch_size=64,
    # Number of steps used for a linear warmup
    warmup_steps=100,                
    weight_decay=0.01,
    # TensorBoard log directory
    logging_strategy='steps',                
    logging_dir='./logs',            
    logging_steps=50,
    # other options : no, steps
    evaluation_strategy="steps",
    save_strategy="epoch",
    fp16=True,
    load_best_model_at_end=True
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [28]:
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

In [29]:
import numpy as np
from scipy.stats import pearsonr
from scipy.stats import spearmanr
def compute_metrics(pred):
    preds = np.squeeze(pred.predictions) 
    return {"MSE": ((preds - pred.label_ids) ** 2).mean().item(),
            "RMSE": (np.sqrt ((  (preds - pred.label_ids) ** 2).mean())).item(),
            "MAE": (np.abs(preds - pred.label_ids)).mean().item(),
            "Pearson" : pearsonr(preds,pred.label_ids)[0],
            "Spearman's Rank" : spearmanr(preds,pred.label_ids)[0]
            }

In [30]:
trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=enc_train,
        eval_dataset=enc_val,
        compute_metrics=compute_metrics,
        tokenizer=tokenizer
    )

Using amp fp16 backend


In [31]:
train_result = trainer.train()
metrics = train_result.metrics

The following columns in the training set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: idx, sentence2, sentence1.
***** Running training *****
  Num examples = 5749
  Num Epochs = 3
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 540
/usr/local/lib/python3.7/dist-packages/transformers/trainer.py:1299: FutureWarning: Non-finite norm encountered in torch.nn.utils.clip_grad_norm_; continuing anyway. Note that the default behavior will change in a future release to error out if a non-finite total norm is encountered. At that point, setting error_if_nonfinite=false will be required to retain the old behavior.
  args.max_grad_norm,


Step,Training Loss,Validation Loss,Mse,Rmse,Mae,Pearson,Spearman's rank
50,5.964800,2.136971,2.136971,1.461838,1.242887,0.239567,0.231133
100,1.924600,0.828052,0.828052,0.909974,0.729926,0.796842,0.797176
150,0.979900,0.703802,0.703802,0.838929,0.682661,0.835054,0.832703
200,0.787500,0.658153,0.658153,0.811267,0.623839,0.850082,0.846071
250,0.586400,0.625896,0.625896,0.791136,0.614382,0.858766,0.854464
300,0.527500,0.602254,0.602254,0.776050,0.620157,0.862716,0.858086
350,0.490400,0.565506,0.565506,0.752001,0.582657,0.867851,0.862989
400,0.365000,0.581609,0.581609,0.762633,0.590046,0.867682,0.863059
450,0.321000,0.545002,0.545002,0.738242,0.576358,0.870531,0.865723
500,0.310700,0.548513,0.548513,0.740617,0.578976,0.867558,0.862267


The following columns in the evaluation set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: idx, sentence2, sentence1.
***** Running Evaluation *****
  Num examples = 750
  Batch size = 64
Saving model checkpoint to ./stsb-model/checkpoint-50
Configuration saved in ./stsb-model/checkpoint-50/config.json
Model weights saved in ./stsb-model/checkpoint-50/pytorch_model.bin
tokenizer config file saved in ./stsb-model/checkpoint-50/tokenizer_config.json
Special tokens file saved in ./stsb-model/checkpoint-50/special_tokens_map.json
The following columns in the evaluation set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: idx, sentence2, sentence1.
***** Running Evaluation *****
  Num examples = 750
  Batch size = 64
Saving model checkpoint to ./stsb-model/checkpoint-100
Configuration saved in ./stsb-model/checkpoint-100/config.json
Model weights saved in ./stsb-model/ch

In [32]:
s1,s2="A plane is taking off.",	"An air plane is taking off."

In [33]:
encoding = tokenizer(s1,s2, return_tensors='pt', padding=True, truncation=True, max_length=512)
input_ids = encoding['input_ids'].to(device)
attention_mask = encoding['attention_mask'].to(device)
outputs = model(input_ids, attention_mask=attention_mask)
outputs.logits.item()

4.1813435554504395

In [34]:
s1,s2="The men are playing soccer.",	"A man is riding a motorcycle."

In [35]:
encoding = tokenizer("hey how are you there","hey how are you", return_tensors='pt', padding=True, truncation=True, max_length=512)
input_ids = encoding['input_ids'].to(device)
attention_mask = encoding['attention_mask'].to(device)
outputs = model(input_ids, attention_mask=attention_mask)
outputs.logits.item()

2.4175961017608643

In [36]:
q=[trainer.evaluate(eval_dataset=data) for data in [enc_train, enc_val, enc_test]]
pd.DataFrame(q, index=["train","val","test"]).iloc[:,:6]

The following columns in the evaluation set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: idx, sentence2, sentence1.
***** Running Evaluation *****
  Num examples = 5749
  Batch size = 64


The following columns in the evaluation set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: idx, sentence2, sentence1.
***** Running Evaluation *****
  Num examples = 750
  Batch size = 64
The following columns in the evaluation set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: idx, sentence2, sentence1.
***** Running Evaluation *****
  Num examples = 750
  Batch size = 64


,eval_loss,eval_MSE,eval_RMSE,eval_MAE,eval_Pearson,eval_Spearman's Rank
train,0.263773,0.263773,0.513588,0.398535,0.939193,0.930282
val,0.545002,0.545002,0.738242,0.576358,0.870531,0.865723
test,0.555785,0.555785,0.745510,0.581903,0.872935,0.870873


In [37]:
model_path = "sentence-pair-regression-model"
trainer.save_model(model_path)
tokenizer.save_pretrained(model_path)

Saving model checkpoint to sentence-pair-regression-model
Configuration saved in sentence-pair-regression-model/config.json
Model weights saved in sentence-pair-regression-model/pytorch_model.bin
tokenizer config file saved in sentence-pair-regression-model/tokenizer_config.json
Special tokens file saved in sentence-pair-regression-model/special_tokens_map.json
tokenizer config file saved in sentence-pair-regression-model/tokenizer_config.json
Special tokens file saved in sentence-pair-regression-model/special_tokens_map.json


('sentence-pair-regression-model/tokenizer_config.json',
 'sentence-pair-regression-model/special_tokens_map.json',
 'sentence-pair-regression-model/vocab.txt',
 'sentence-pair-regression-model/added_tokens.json',
 'sentence-pair-regression-model/tokenizer.json')